In [1]:
import numpy as np
import emcee
import corner
import pymc3 as pm
from scipy import stats
from scipy import optimize
from scipy import integrate
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Dataset for this project (Obtained from Pitkin (https://github.com/mattpitkin/periodicG))
Ggrad = (6.676e-11-6.672e-11)/(372.31-96.28)
UWup = 6.672e-11 + Ggrad*(249.17-96.28)
UWuperr = (316.94-249.17)*Ggrad
G = np.array([6.67248e-11, 6.67398e-11, 6.67228e-11, 6.674255e-11, 6.67559e-11, UWup, 6.67387e-11, 6.67234e-11, 6.674252e-11, 6.67554e-11, 6.67349e-11, 6.67191e-11])
Err = np.array([0.00043e-11, 0.00070e-11, 0.00087e-11, 0.000092e-11, 0.00027e-11, UWuperr, 0.00027e-11, 0.00014e-11, 0.000120e-11, 0.00016e-11, 0.00018e-11, 0.00099e-11])
Year = np.array([1981.90,1996.97,1998.32,2000.46,2001.16,2002.02,2003.39,2004.40,2006.48,2007.68,2009.17,2013.57])
relYear = Year - Year[0]

posgrad = (2015.-1981.) / (674.04-92.79)
positions = np.array([108.17, 365.87, 388.94, 425.48, 437.50, 452.08, 475.48, 492.79, 528.44, 552.61, 574.52, 649.52])
positions = positions-92.79
years = positions*posgrad
t_0 = (420.83-92.79)*posgrad

In [3]:
timeserrs = (3./12.)*np.ones_like(Err) # three month errors
timeserrs[7] = (1./52.) # one week error on JILA-10
timeserrs[11] = (1./52.) # one week error on LENS-14

In [4]:
# Range of parameters
meanG = np.mean(G)
sigma_meanG = np.std(G) / np.sqrt(len(G))  
muGmin = meanG - 6.*sigma_meanG
muGmax = meanG + 6.*sigma_meanG

sigmasysmax = np.max(G)-np.min(G)
sigmasysmin = np.min(Err)

Amin = sigmasysmin
Amax = sigmasysmax

timediff = np.diff(np.sort(years)[1:]) # skip the first longer time

periodmin = 5.90
periodmax = 5.90

phimin = 0
phimax = 0

priorSet = [muGmin,muGmax,sigmasysmin,sigmasysmax,Amin,Amax,periodmin,periodmax,phimin,phimax]
data = [G,Err,years]

In [5]:
def sigmoid(x) : 
    return 1 / ( 1 + np.exp(-x))
'''
 In ADVI, the variational distribution is taken as gaussian with diagonal variance. p(x) ~ N(mu,std^2)
'''
def variational_pdf(x, mu, std, b, a) : 
    return (stats.norm.pdf(x, mu, std) / ((b - a) * sigmoid(x) * (1 - sigmoid(x)))).sum()

# Hyp 1

In [6]:
with pm.Model() as hyp_1:
    muG = pm.Uniform('muG', lower=muGmin, upper=muGmax)
    y,sigma_y,time = data
    mu = np.repeat(muG, len(y))
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma_y,observed=y)

with hyp_1:
    approx1 = pm.fit(n=100000,method='advi',obj_optimizer=pm.adam(learning_rate=1e-4),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute',tolerance=1e-4)])
    trace1 = pm.sample_approx(approx1)


Average Loss = -232.14:  93%|█████████▎| 92599/100000 [00:13<00:01, 6912.19it/s]
Convergence achieved at 92600
Interrupted at 92,599 [92%]: Average Loss = -223.81


In [7]:
# get variational parameters in unconstrained space
mu = approx1.mean.eval()
std = approx1.std.eval()

print('mean : ', mu)
print('std : ', std)

mean :  [0.35940277]
std :  [0.05447173]


In [8]:
def hyp_1_logp(muG):
    log_p = 0.
    log_p += stats.uniform.logpdf(muG,muGmin,muGmax - muGmin)
    mu = np.repeat(muG, len(y))
    log_p += stats.norm.logpdf(y,mu,sigma_y).sum()
    return log_p

hyp_1_logp_vec = np.vectorize(hyp_1_logp)

In [9]:
approx_evidence = (np.exp(hyp_1_logp_vec(trace1['muG'])) / variational_pdf(trace1['muG_interval__'],mu,std,muGmax,muGmin)).mean()
print('Approx evidence : ', approx_evidence)
print('log approx evidence : ', np.log(approx_evidence))

Approx evidence :  6.53969331876992e+98
log approx evidence :  227.53122938463162


In [10]:
elbo1 = (hyp_1_logp_vec(trace1['muG']) - np.log(variational_pdf(trace1['muG_interval__'],mu,std,muGmax,muGmin))).mean()
print(elbo1)

227.34375660626367


# Hyp 2

In [11]:
with pm.Model() as hyp_2:
    muG = pm.Uniform('muG', lower=muGmin, upper=muGmax)
    sigmasys = pm.Uniform('sigmasys',lower=sigmasysmin,upper=sigmasysmax)
    mu = np.repeat(muG, len(y))
    y,sigma_y,time = data
    sd = np.sqrt(sigma_y**2 + sigmasys**2)
    y_obs = pm.Normal('y_obs',mu=mu,sd=sd,observed=y)
with hyp_2:
    approx2 = pm.fit(n=400000,method='advi',obj_optimizer=pm.adam(learning_rate=1e-4),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute',tolerance=1e-4)])
    trace2 = pm.sample_approx(approx2)

Average Loss = -364.67: 100%|██████████| 400000/400000 [01:03<00:00, 6268.92it/s]
Finished [100%]: Average Loss = -364.67


In [12]:
means_dict = approx2.bij.rmap(approx2.params[0].eval())
rho_dict = approx2.bij.rmap(approx2.params[1].eval())
std_dict ={k: np.log(1 + np.exp(v)) for k, v in rho_dict.items()}

print(means_dict)
print(std_dict)

{'sigmasys_interval__': array(-0.73748697), 'muG_interval__': array(0.1038305)}
{'sigmasys_interval__': 0.3752807466229987, 'muG_interval__': 0.3751976887136172}


In [13]:
def hyp_2_logp(trace):
    log_p = 0.
    log_p += stats.uniform.logpdf(trace['muG'],muGmin,muGmax - muGmin)
    log_p += stats.uniform.logpdf(trace['sigmasys'],sigmasysmin,sigmasysmax - sigmasysmin)
    mu = np.repeat(trace['muG'], len(y))
    sd = np.sqrt(sigma_y**2 + trace['sigmasys']**2)
    log_p += stats.norm.logpdf(y,mu,sd).sum()
    return log_p

hyp_2_logp_vec = np.vectorize(hyp_2_logp)

In [14]:
def q(trace,model):
    qw=1.
    l=muGmin
    u=muGmax
    for var in model.vars:
        var = str(var).split('~')[0].strip()
        #print(str(var))
        if(str(var)[:len(str(var)) - len("_interval__")]) == "muG":
            l=muGmin
            u=muGmax
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'sigmasys':
            l = sigmasysmin
            u = sigmasysmax
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'A':
            l = Amin
            u = Amax
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'phi':
            l = 0
            u = 2*np.pi
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'ts':
            l = years - 2.5*timeserrs
            u = years
        elif (str(var)[:len(str(var)) - len("_interval__")]) == 'P':
            l = periodmin
            u = periodmax
        qw *= variational_pdf(trace[str(var)],means_dict[str(var)], std_dict[str(var)],u,l)
        #print(qw)
    return qw

q_vec = np.vectorize(q)

In [15]:
approx_evidence2 = (np.exp(hyp_2_logp_vec(trace2)) / q_vec(trace2,hyp_2)).mean()
print('Approx evidence : ', approx_evidence2)
print('log approx evidence : ', np.log(approx_evidence2))

Approx evidence :  2.3712779876432093e+158
log approx evidence :  364.67187373816364


In [16]:
elbo2 = (hyp_2_logp_vec(trace2) - np.log(q_vec(trace2,hyp_2))).mean()
print(elbo2)

364.62337857289634


# Hyp 3

In [17]:
timediff = np.diff(np.sort(years)[1:])
periodmin = 2*np.min(timediff)
periodmax = np.max(timediff)

In [18]:
with pm.Model() as hyp_3:
    y,sigma_y,time = data
    muG = pm.Uniform('muG', lower=muGmin, upper=muGmax)
    A = pm.Uniform('A', lower=Amin,upper=Amax)
    phi = pm.Uniform('phi',lower=0.,upper=2*np.pi)
    #ts = pm.Uniform('ts',lower=time - 2.5*timeserrs,upper=time,shape=len(time))
    P = pm.Uniform('P', lower=periodmin,upper=periodmax)
    mu = muG + A * np.sin(phi + 2 * np.pi * time / P)
    y_obs = pm.Normal('y_obs',mu=mu,sd=sigma_y,observed=y)
with hyp_3:
    approx3 = pm.fit(n=400000,method='advi',obj_optimizer=pm.adam(learning_rate=1e-4),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute',tolerance=1e-4)])
    trace3 = pm.sample_approx(approx3)

Average Loss = -226.25: 100%|██████████| 400000/400000 [01:07<00:00, 5913.68it/s]
Finished [100%]: Average Loss = -226.15


In [19]:
means_dict = approx3.bij.rmap(approx3.params[0].eval())
rho_dict = approx3.bij.rmap(approx3.params[1].eval())
std_dict ={k: np.log(1 + np.exp(v)) for k, v in rho_dict.items()}

print(means_dict)
print(std_dict)

{'A_interval__': array(-4.82457626), 'phi_interval__': array(-0.19722192), 'P_interval__': array(-0.01651674), 'muG_interval__': array(0.35800202)}
{'A_interval__': 0.8368647355258746, 'phi_interval__': 0.8489322223055673, 'P_interval__': 0.816633799550027, 'muG_interval__': 0.05483018600694995}


In [20]:
def hyp_3_logp(trace):
    log_p = 0.
    log_p += stats.uniform.logpdf(trace['muG'],muGmin,muGmax - muGmin)
    log_p += stats.uniform.logpdf(trace['A'],Amin,Amax - Amin)
    log_p += stats.uniform.logpdf(trace['phi'],0,2*np.pi)
    log_p += stats.uniform.logpdf(trace['P'],periodmin,periodmax-periodmin)
    mu = trace['muG'] + trace['A'] * np.sin(trace['phi'] + 2 * np.pi * time / trace['P'])
    log_p += stats.norm.logpdf(y,mu,sigma_y).sum()
    return log_p

hyp_3_logp_vec = np.vectorize(hyp_3_logp)

In [21]:
q_vec(trace3[0],hyp_3)

array(5.38772646e+27)

In [22]:
approx_evidence3 = (np.exp(hyp_3_logp_vec(trace3)) / q_vec(trace3,hyp_3)).mean()
print('Approx evidence : ', approx_evidence3)
print('log approx evidence : ', np.log(approx_evidence3))

Approx evidence :  2.4369911755064877e+110
log approx evidence :  254.17512438293372


In [23]:
elbo3 = (hyp_3_logp_vec(trace3) - np.log(q_vec(trace3,hyp_3))).mean()
print(elbo3)

226.98970830521176


# hyp 4

In [24]:
with pm.Model() as hyp_4:
    y,sigma_y,time = data
    muG = pm.Uniform('muG', lower=muGmin, upper=muGmax)
    A = pm.Uniform('A', lower=Amin,upper=Amax)
    phi = pm.Uniform('phi',lower=0.,upper=2*np.pi)
    sigmasys = pm.Uniform('sigmasys',lower=sigmasysmin,upper=sigmasysmax)
    P = pm.Uniform('P', lower=periodmin,upper=periodmax)
    mu = muG + A * np.sin(phi + 2 * np.pi * time / P)
    sd = np.sqrt(sigma_y**2 + sigmasys**2)
    y_obs = pm.Normal('y_obs',mu=mu,sd=sd,observed=y)
with hyp_4:
    approx4 = pm.fit(n=400000,method='advi',obj_optimizer=pm.adam(learning_rate=1e-4),callbacks=[pm.callbacks.CheckParametersConvergence(diff='absolute',tolerance=1e-4)])
    trace4 = pm.sample_approx(approx4)

Average Loss = -362.7: 100%|██████████| 400000/400000 [01:11<00:00, 5620.95it/s] 
Finished [100%]: Average Loss = -362.7


In [25]:
means_dict = approx4.bij.rmap(approx4.params[0].eval())
rho_dict = approx4.bij.rmap(approx4.params[1].eval())
std_dict ={k: np.log(1 + np.exp(v)) for k, v in rho_dict.items()}

print(means_dict)
print(std_dict)

{'sigmasys_interval__': array(-0.61789422), 'muG_interval__': array(0.09918406), 'A_interval__': array(-2.36018853), 'phi_interval__': array(-0.13348387), 'P_interval__': array(0.01864589)}
{'sigmasys_interval__': 0.38388456863550247, 'muG_interval__': 0.4033109446084607, 'A_interval__': 0.9587299824571236, 'phi_interval__': 1.4958199656598883, 'P_interval__': 1.494047928430552}


In [26]:
def hyp_4_logp(trace):
    log_p = 0.
    log_p += stats.uniform.logpdf(trace['muG'],muGmin,muGmax - muGmin)
    log_p += stats.uniform.logpdf(trace['A'],Amin,Amax - Amin)
    log_p += stats.uniform.logpdf(trace['phi'],0,2*np.pi)
    log_p += stats.uniform.logpdf(trace['P'],periodmin,periodmax-periodmin)
    log_p += stats.uniform.logpdf(trace['sigmasys'],sigmasysmin,sigmasysmax - sigmasysmin)
    mu = trace['muG'] + trace['A'] * np.sin(trace['phi'] + 2 * np.pi * time / trace['P'])
    sd = np.sqrt(sigma_y**2 + trace['sigmasys']**2)
    log_p += stats.norm.logpdf(y,mu,sd).sum()
    return log_p

hyp_4_logp_vec = np.vectorize(hyp_4_logp)

In [27]:
approx_evidence4 = (np.exp(hyp_4_logp_vec(trace4)) / q_vec(trace4,hyp_4)).mean()
print('Approx evidence : ', approx_evidence4)
print('log approx evidence : ', np.log(approx_evidence4))

Approx evidence :  6.800917123103344e+157
log approx evidence :  363.4229170741976


In [28]:
elbo4 = (hyp_4_logp_vec(trace4) - np.log(q_vec(trace4,hyp_4))).mean()
print(elbo4)

362.9354069309452


# nestle

In [29]:
import nestle

In [30]:
%%time
def loglike_1(P):
    mu = np.repeat(P[0], len(y))
    return sum(stats.norm.logpdf(*args) for args in zip(y,mu,sigma_y))

def prior_transform_1(P):
    return np.array([(muGmax - muGmin)*P[0]+muGmin])

result_1 = nestle.sample(loglike_1, prior_transform_1, 1)

print(result_1.logz)     # log evidence
print(result_1.logzerr)

232.10272646913808
0.17319730047704412
CPU times: user 788 ms, sys: 33 ms, total: 821 ms
Wall time: 791 ms


In [31]:
%%time
def loglike_2(P):
    mu = np.repeat(P[0], len(y))
    sd = np.sqrt(sigma_y**2 + P[1]**2)
    return sum(stats.norm.logpdf(*args) for args in zip(y,mu,sd))

def prior_transform_2(P):
    return np.array([(muGmax - muGmin)*P[0] + muGmin, (sigmasysmax - sigmasysmin) * P[1] + sigmasysmin])

result_2 = nestle.sample(loglike_2, prior_transform_2, 2)

print(result_2.logz)     # log evidence
print(result_2.logzerr)

364.846311269481
0.13742716983384742
CPU times: user 790 ms, sys: 4.1 ms, total: 794 ms
Wall time: 798 ms


In [32]:
%%time
def loglike_3(P):
    mu = P[0] + P[1] * np.sin(P[2] + 2 * np.pi * time / P[3])
    return sum(stats.norm.logpdf(*args) for args in zip(y,mu,sigma_y))

def prior_transform_3(P):
    return np.array([(muGmax - muGmin)*P[0] + muGmin,
                    (Amax - Amin) * P[1] + Amin,
                    2*np.pi * P[2], 
                    (periodmax - periodmin) * P[3] + periodmin])

result_3 = nestle.sample(loglike_3, prior_transform_3, 4)
print(result_3.logz)     # log evidence
print(result_3.logzerr)  # numerical (sampling) error on logz

313.5903768451409
0.3447761287045049
CPU times: user 13min 4s, sys: 2.67 s, total: 13min 7s
Wall time: 13min 2s


In [33]:
%%time
def loglike_4(P):
    mu = P[0] + P[1] * np.sin(P[2] + 2 * np.pi * time / P[3])
    sd = np.sqrt(sigma_y**2 + P[4]**2)
    return sum(stats.norm.logpdf(*args) for args in zip(y,mu,sd))

def prior_transform_4(P):
    return np.array([(muGmax - muGmin)*P[0] + muGmin,
                    (Amax - Amin) * P[1] + Amin,
                    2*np.pi * P[2], 
                    (periodmax - periodmin) * P[3] + periodmin,
                    (sigmasysmax - sigmasysmin) * P[4] + sigmasysmin])

result_4 = nestle.sample(loglike_4, prior_transform_4, 5)
print(result_4.logz)     # log evidence
print(result_4.logzerr)  # numerical (sampling) error on logz

363.5889785652872
0.18057415227619145
CPU times: user 15.5 s, sys: 49.6 ms, total: 15.6 s
Wall time: 15.5 s


In [34]:
# %%time
# import numpy as np
# import dynesty
# from dynesty import plotting as dyplot


# def loglike_3(P):
#     mu = P[0] + P[1] * np.sin(P[2] + 2 * np.pi * time / P[3])
#     return sum(stats.norm.logpdf(*args) for args in zip(y,mu,sigma_y))

# def prior_transform_3(P):
#     return np.array([(muGmax - muGmin)*P[0] + muGmin,
#                     (Amax - Amin) * P[1] + Amin,
#                     2*np.pi * P[2], 
#                     (periodmax - periodmin) * P[3] + periodmin])


# # Sample from our distribution.
# sampler = dynesty.NestedSampler(loglike_3, prior_transform_3, 4,
#                                 bound='single', nlive=1000)
# sampler.run_nested(dlogz=0.01)
# res = sampler.results